# Détectin des Risques de Churn Client avec un Modele de Classification Supervisée

#### Projet par groupe

### Membre du groupe:
#### * AMKHIB Fatima-Zohra
#### * BENALLAL MEGHOGHI Mohamed Amine
#### * KHERIBI Nour
#### * WAHABI Wissal

# Exploration du Data

In [58]:
import pandas as pd

AttributeError: partially initialized module 'pandas' has no attribute '_pandas_parser_CAPI' (most likely due to a circular import)

In [ ]:
df= pd.read_csv('Churn_Modelling.csv')
df

# Nettoyage du Data

In [ ]:
df.isnull().sum()

#### ==> Cette etape montre qu'il n'y a pas des valeurs manquante

In [ ]:
df.duplicated().sum()

#### ==> Pas de valeurs dupliquees 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns
sns.boxplot(df)
plt.show()

In [ ]:
sns.boxplot(df['Age'])
plt.show()

In [ ]:
sns.boxplot(df['CreditScore'])
plt.show()

In [ ]:
df.describe()

#### ==> il y a des outliers dans la colonne 'Age' et 'CreditScore' mais c'est logique

In [ ]:
df['Geography'].value_counts()

# Suppression des variables inutile

In [ ]:
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)

In [ ]:
df

# Visualilation du Data

## distribuation du geography

In [ ]:
sns.histplot(df['Geography'], bins=5, kde=True, color='blue')
plt.show()


## Distribution du genre

In [ ]:
df['Gender'].value_counts()

In [ ]:
gender = ['male', 'female']
x = [5457, 4543]

plt.pie(x, labels = gender, autopct = '%1.1f%%')
plt.title('Gender Distribution')
plt.show()

## Distribution des variables numeriques

In [ ]:
df.hist(bins=11, figsize=(20,15))
plt.show()

In [ ]:
df

## Distribution du variable cible

In [ ]:
df['Exited'].value_counts()

In [ ]:
Exited = [0, 1]
x = [7963, 2037]

plt.pie(x, labels = Exited, autopct = '%1.1f%%')
plt.title('Exited Distribution')
plt.show()

#### ==> On peut remarquer que la dataset n'est pas équilibrer

In [ ]:
d = df.groupby('Gender')['Balance'].mean()
x = d.values
y = d.index

plt.pie(x, labels = y, autopct='%1.1f%%')
plt.show()

In [ ]:
f = df.groupby('Geography')['Gender'].value_counts()
f


# Encodage des colonnes categorielles

In [ ]:
from sklearn.preprocessing import LabelEncoder

df['Gender']=LabelEncoder().fit_transform(df['Gender'])
df['Geography']=LabelEncoder().fit_transform(df['Geography'])

In [ ]:
df

## Visualisation de la relation entre les caractéristiques et la colonne cible

In [ ]:
sns.pairplot(df, hue='Exited')
plt.show()

# Séparer data en X et Y

In [ ]:
x= df.drop(columns='Exited')
y= df['Exited']

In [ ]:
x.shape

# Normalisation des données

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x= scaler.fit_transform(x)

# Equlibrage du Dataset

In [ ]:
from imblearn.combine import SMOTEENN

smote = SMOTEENN()
X_res, y_res = smote.fit_resample(x, y)

In [ ]:
X_res

# Diviser en train/test

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

# Logistique regression

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(x_train, y_train)

In [ ]:
y_pred= model.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

precision= precision_score(y_test, y_pred, average='macro')
print("Precision:", precision)

#### ==> L'accuracy est un petit peut faible 

## Cross validation 

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
kf= KFold(n_splits=5, shuffle=True, random_state=42)
scores= cross_val_score(model,x,y,cv=kf)
print(f"scores de validation croisee: {scores}")
print(f"score moyen: {scores.mean(): 2f}")

#### ==> On utilisons la cross validation l'accuracy augmente à 80%

## Matrice de confusion

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe0', 'Classe1'], yticklabels=['Classe 0', 'Classe 1'])
plt.xlabel('Prédit')
plt.ylabel('Réel')
plt.title('Matrice de Confusion')
plt.show()

## ROC-AUC

In [ ]:
from sklearn.metrics import roc_curve, auc


# Calcul de la courbe ROC
y_scores = model.predict_proba(x_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)

# Affichage de la courbe ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Courbe ROC')
plt.legend()
plt.show()

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn= KNeighborsClassifier(n_neighbors=3)
knn.fit(x_train, y_train)

In [ ]:
y_pred_knn= knn.predict(x_test)

In [ ]:
accuracy_knn= accuracy_score(y_test, y_pred_knn)
accuracy_knn

#### ==> L'accuracy avec KNN est à propos de 98%, c-a-d que le model a bien s'entrainer

## Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
kf= KFold(n_splits=5, shuffle=True, random_state=42)
scores= cross_val_score(knn,x,y,cv=kf)
print(f"scores de validation croisee: {scores}")
print(f"score moyen: {scores.mean(): 2f}")

## Matrice de Confusion

In [ ]:
confu_matrix = confusion_matrix(y_test, y_pred_knn)
sns.heatmap(confu_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe0', 'Classe1'], yticklabels=['Classe 0', 'Classe 1'])
plt.xlabel('Prédit')
plt.ylabel('Réel')
plt.title('Matrice de Confusion')
plt.show()

#### ==> La matrice de confusion nous montre que le model fait la majorité des prédiction juste

## ROC-AUC

In [ ]:
# Calcul de la courbe ROC
y_scores = knn.predict_proba(x_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)

# Affichage de la courbe ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Courbe ROC')
plt.legend()
plt.show()

### ==> Cette courbe ROC montre que la marge d'erreur est tros faible dans ce model 

# Decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

tree = DecisionTreeClassifier(criterion="gini", max_depth=3, random_state=42)
tree.fit(x_train, y_train)

print("Modèle d'Arbre de Décision entraîné avec succès.")

In [ ]:
y_pred_tree= tree.predict(x_test)
accuracy_TREE= accuracy_score(y_test, y_pred_tree)
print("Laccuracy de l'arbre de decision est", accuracy_TREE)

#### ==> On utilisons 'Decision tree', l'accurcy est de 80% (moins que KNN)

## Cross de Validation

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
kf= KFold(n_splits=5, shuffle=True, random_state=42)
scores= cross_val_score(tree,x,y,cv=kf)
print(f"scores de validation croisee: {scores}")
print(f"score moyen: {scores.mean(): 2f}")

## Matrice de Confusion

In [ ]:
confus_matrix = confusion_matrix(y_test, y_pred_tree)
sns.heatmap(confus_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe0', 'Classe1'], yticklabels=['Classe 0', 'Classe 1'])
plt.xlabel('Prédit')
plt.ylabel('Réel')
plt.title('Matrice de Confusion')
plt.show()

#### ==> Cette matrice de confusion nous montre que le model fait des fauttes pour prédire la classe 1

## Visualisation de l'arbre de decision 

In [ ]:
plt.figure(figsize=(30, 20))
plot_tree(
    tree,
    filled=True,
    rounded=True,
    fontsize=18
)
plt.title("Visualisation de l'Arbre de Décision")
plt.show()

## ROC-AUC

In [ ]:
# Calcul de la courbe ROC
y_scores = tree.predict_proba(x_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)

# Affichage de la courbe ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Courbe ROC')
plt.legend()
plt.show()

# SVM

In [ ]:
from sklearn.svm import SVC

svc_model = SVC(kernel='rbf') 
svc_model.fit(x_train, y_train)

In [ ]:
y_pred_svc = svc_model.predict(x_test)


In [ ]:
accuracy_svc= accuracy_score(y_test, y_pred_svc)
accuracy_svc

#### ==> Avec 'SVM', l'accuracy est 91% mais KNN reste mieux

## Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
kf= KFold(n_splits=5, shuffle=True, random_state=42)
scores= cross_val_score(svc_model,x,y,cv=kf)
print(f"scores de validation croisee: {scores}")
print(f"score moyen: {scores.mean(): 2f}")

## Matrice de confusion

In [ ]:
confusi_matrix = confusion_matrix(y_test, y_pred_svc)
sns.heatmap(confusi_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Prédit')
plt.ylabel('Réel')
plt.title('Matrice de Confusion')
plt.show()

#### ==> La matrice de confusion de 'SVM', montre que le model prédit malle la classe 0

In [ ]:
# # Calcul de la courbe ROC
# y_scores = svc_model.predict_proba(x_test)[:, 1]
# fpr, tpr, _ = roc_curve(y_test, y_scores)
# roc_auc = auc(fpr, tpr)

# # Affichage de la courbe ROC
# plt.figure(figsize=(8, 6))
# plt.plot(fpr, tpr, color='blue', label=f'AUC = {roc_auc:.2f}')
# plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Courbe ROC')
# plt.legend()
# plt.show()

# Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=1000, random_state=42)
rf.fit(x_train, y_train)

In [ ]:
y_predr = rf.predict(x_test)
accuracyr = accuracy_score(y_test, y_predr)
print(f'Accuracy du model RF : {accuracyr: .2f}')

#### ==> L'accuracy est proche à celle de KNN c-a-d le modele avec Random forest est aussi bien pour la prédiction

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print('Rapport de classification: ')
print(classification_report(y_test, y_predr, target_names=[str(cls) for cls in rf.classes_]))

In [ ]:
confusi_matrix = confusion_matrix(y_test, y_predr)
sns.heatmap(confusi_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe0', 'Classe1'], yticklabels=['Classe 0', 'Classe 1'])
plt.xlabel('Prédit')
plt.ylabel('Réel')
plt.title('Matrice de Confusion')
plt.show()

#### ==> La matrice de confusion est bonne pour une prédiction avec une marge d'erreur équilibrer entre les deux classes

In [ ]:
# Calcul de la courbe ROC
y_scores = rf.predict_proba(x_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)

# Affichage de la courbe ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Courbe ROC')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

def plot_roc(model, x_test, y_test, title):
    if hasattr(model, "predict_proba"):
        y_prob = model.predict_proba(x_test)[:, 1]
    else:
        y_prob = model.decision_function(x_test)
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    auc = roc_auc_score(y_test, y_prob)

    plt.plot(fpr, tpr, label=f'{title} (AUC = {auc:.2f})')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()

# Affichage ROC
plt.figure(figsize=(6, 5))
plt.plot([0, 1], [0, 1], "k--")
plot_roc(knn, x_test, y_test, "KNN")
plot_roc(svc_model, x_test, y_test, "SVM")
plot_roc(tree, x_test, y_test, "Decision Tree")
plot_roc(model, x_test, y_test, "Logistic Regression")
plot_roc(rf, x_test, y_test, "Random Forest")
plt.grid()
plt.show()

L'accuracy des modeles:

- Logistic regression: 77%
- Knn: 98%
- Decision tree: 84%
- SVM: 85%
- Random forest: 96%

### ==> On peut remarquer que les deux modeles (KNN, Random forest) sont proches au niveaux d'accuracy, mais on s'appuiant sur aussi sur la matrice de confusion, on décider que le model KNN est le plus précis

## Prediction

In [ ]:
pred=knn.predict([[0.30, 0.04, 1.80, 0.8, 0.5, 1.35, 0.1, 1, 0, 0.66]])
pred

#### ==> On a choisie le modele KNN comme le meilleur modele pour la prediction, car il a obtenu la meilleure precision (99%) et la meilleure accuracy (98%).

In [ ]:
print('Rapport de classification: ')
print(classification_report(y_test, y_pred_knn, target_names=[str(cls) for cls in knn.classes_]))

# Enregistrement du meilleur model

import joblib
joblib.dump(knn, 'knn_churn.pkl')